# Final Projet - BobbyBot

In [15]:
import pandas as pd
import numpy as np
import string ,re , random
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import warnings
import speech_recognition as sr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import playsound
import os
import random
from gtts import gTTS
import pyttsx3
import pyaudio
import time
import random
import gensim
import spacy
from spacy.matcher import Matcher

from spacy import displacy 
from pprint import pprint

nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')
porter = nltk.PorterStemmer()
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nohra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nohra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('tennis.txt', delimiter = "\t", names =['text'])

In [3]:
df.head()

,text
0,Predecessors
1,Jeu de paume in the 17th century
2,Historians believe that the game's ancient ori...
3,It was not until the 16th century that rackets...
4,"An epitaph in St Michael's Church, Coventry, w..."


## Preprocessing

In order to work with the data, we have cleaned it from all URLs, stemmed the words and removed stopwords

In [4]:
def cleaning_URLs(text):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ', text)
def Stemming_tweets(tweet):
    return [porter.stem(w) for w in tweet]
def cleaning_stopwords(text):
    return" ".join(word for word in str(text).split() if word not in stop_words)

df['cleaned_text'] = df['text'].str.replace(r'[^0-9a-zA-Z]+',' ')
df['cleaned_text'] = df['cleaned_text'].apply(lambda msg: cleaning_URLs(msg))
df['cleaned_text'] = df['cleaned_text'].apply(lambda tw: cleaning_stopwords(tw))
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: word_tokenize(x))
df['cleaned_text'] = df['cleaned_text'].apply(lambda tw: Stemming_tweets(tw))

df.head()

,text,cleaned_text
0,Predecessors,[predecessor]
1,Jeu de paume in the 17th century,"[jeu, de, paum, 17th, centuri]"
2,Historians believe that the game's ancient ori...,"[historian, believ, game, ancient, origin, lay..."
3,It was not until the 16th century that rackets...,"[It, 16th, centuri, racket, came, use, game, b..."
4,"An epitaph in St Michael's Church, Coventry, w...","[An, epitaph, St, michael, church, coventri, w..."


In [6]:
#here we have combined Module 2 and 3 
#introducing our lists for the user's greetings and the bot reply
#we also defined the function for greeting between user and bot 

my_greetings = ['hi', 'hi there', 'hey', 'hello', 'hello bobby bot','hello Bobby bot']
bot_greetings = ['Heyy', 'what\'s up bro!', 'Helloz, Nice to see you Again', 'Hello Dear, enjoy your session']

def greeting(question):
    if question in my_greetings:
        response = random.choice(bot_greetings)
    else:
        response = 'I am sorry, I do not understand what you are asking me.'
    return response

In [7]:
#converted the cleaned tokens into text

text = []

for word in df['cleaned_text']:
    sent = " ".join(word)
    text.append(sent)

In [8]:
#defined the response function to our question which would find the similarity between the keywords of the question
#and the tennis text

tv = TfidfVectorizer()

def response(question):
    question_vector = tv.fit_transform([question])
    answer_vectors = tv.transform(text)
    similarity = cosine_similarity(question_vector,answer_vectors)
    index = similarity.argsort()[0][-1]
    flattened_similarity = similarity.flatten()
    flattened_similarity.sort()
    value = flattened_similarity[-1]
    if (value == 0):
        response =  "I am sorry, I do not understand what you are asking me"
    else:
        response = df['text'][index]
    return response

In [9]:
#example for the result provided by the fucntion

response('origins of the modern tennis?')

'In Tennis: A Cultural History, Heiner Gillmeister reveals that on 8 December 1874, British army officer Walter Clopton Wingfield wrote to Harry Gem, commenting that he (Wingfield) had been experimenting with his version of lawn tennis “for a year and a half”.[14] In December 1873, Wingfield designed and patented a game which he called sphairistikè (Greek: σφαιριστική, meaning "ball-playing"), and was soon known simply as "sticky" – for the amusement of guests at a garden party on his friend\'s estate of Nantclwyd Hall, in Llanelidan, Wales.[15] According to R. D. C. Evans, turfgrass agronomist, "Sports historians all agree that [Wingfield] deserves much of the credit for the development of modern tennis."[9][16] According to Honor Godfrey, museum curator at Wimbledon, Wingfield "popularized this game enormously. He produced a boxed set which included a net, poles, rackets, balls for playing the game – and most importantly you had his rules. He was absolutely terrific at marketing and 

In [10]:
response('Wimbledon')

'Tennis also became popular in France, where the French Championships dates to 1891 although until 1925 it was open only to tennis players who were members of French clubs.[25] Thus, Wimbledon, the US Open, the French Open, and the Australian Open (dating to 1905) became and have remained the most prestigious events in tennis.[26][27] Together these four events are called the Majors or Slams (a term borrowed from bridge rather than baseball).[28]'

In [11]:
#this code would combine both the input and the output of the user and the bot
#flag would allow the code to run continuously until it is interrupted

flag = True

print("BobbyBot: My name is BobbyBot. I will answer your queries about Tennis. If you want to exit, type exit! ")

while (flag == True):
    question = input()
    question = question.lower()
    
    if (question != 'exit'):
        if (question == 'thanks' or question == 'thank you'):
            flag = False
            print("BobbyBot: You are welcome!")
        else:
            if (question in my_greetings):
                print("BobbyBot: "+ greeting(question))
            else:
                print("BobbyBot: ",end="")
                print(response(question))
    else:
        flag = False
        print("BobbyBot: Bye! take care!")

BobbyBot: My name is BobbyBot. I will answer your queries about Tennis. If you want to exit, type exit! 
exit
BobbyBot: Bye! take care!
